In [1]:
import os

from cartopy import crs
import distributed
from matplotlib import pyplot as plt
import pandas as pd
import xarray as xr

_ = xr.set_options(keep_attrs=True)

In [2]:
# use a large filesystem for cahce
#
# os.chdir("/data")

# activate dask distributed
#
# client = distributed.Client("tcp://dask-scheduler:8786")
# client

In [3]:
# !pip install ecmwf-opendata 
# !pip install -e ../

In [21]:
request = {
    "source": "ecmwf",
    "type": "pf",
    "param": ["2t"],
    "date": -1,
    "step": ["12", "24", "48"],
    "time": ["0", "12"],
    "number": [1, 2],
}

In [28]:
%%time

ds = xr.open_dataset(
    request,
    engine="ecmwf",
    client="ecmwf-opendata",
    request_chunks={"step": 1, "number": 1, "day": 1},
    cache_kwargs={"cache_file": True},
    chunks={},
)
ds

/src/xarray-ecmwf/xarray_ecmwf/client_cdsapi.py:102: UserWarning: Converting non-nanosecond precision timedelta values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  coords["step"] = xr.IndexVariable(  # type: ignore


CPU times: user 431 ms, sys: 1.02 ms, total: 432 ms
Wall time: 434 ms


<xarray.Dataset>
Dimensions:            (step: 3, number: 2, time: 2, latitude: 451,
                        longitude: 900)
Coordinates:
  * number             (number) int32 1 2
  * time               (time) datetime64[ns] 2023-09-28 2023-09-28T12:00:00
  * step               (step) timedelta64[ns] 0 days 12:00:00 ... 2 days
    heightAboveGround  float64 ...
    valid_time         (time) datetime64[ns] dask.array<chunksize=(2,), meta=np.ndarray>
  * latitude           (latitude) float64 90.0 89.6 89.2 ... -89.2 -89.6 -90.0
  * longitude          (longitude) float64 -180.0 -179.6 -179.2 ... 179.2 179.6
Data variables:
    2t                 (step, number, time, latitude, longitude) float32 dask.array<chunksize=(1, 1, 2, 451, 900), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-09-29T21:09 GRIB to CDM+CF via cfgrib-0.9.1...

In [29]:
# visualisation

italy = dict(lat=slice(35, 47), lon=slice(6, 19))
vmax = 0.13
projection = crs.EuroPP()
cmap = "Blues"

In [ ]:
%%time

f, ax = plt.subplots(subplot_kw={"projection": projection})
ds.data_vars["2t"].isel(time=0, step=1, number=1).plot(ax=ax, vmin=273.15, vmax=273.15 + 30, transform=crs.PlateCarree())
_ = ax.coastlines()

In [ ]:
%%time

f, ax = plt.subplots(subplot_kw={"projection": projection})
ds.total_precipitation.sel(time=["2023-05-17T00:00", "2023-05-18T00:00"]).sum("time").plot(ax=ax, cmap=cmap, vmax=vmax, transform=crs.PlateCarree())
_ = ax.coastlines()

In [ ]:
%%time

f, ax = plt.subplots(subplot_kw={"projection": projection})
ds.total_precipitation.sel(time=pd.date_range("2022-05-01", "2022-05-31", freq="D")).sum("time").plot(ax=ax, cmap=cmap, vmax=vmax, transform=crs.PlateCarree())
_ = ax.coastlines()